# Transitive and Intransitive Verbs

## Transitive and Intransitive

In [1]:
from allennlp_models.pretrained import load_predictor

/home/elena/anaconda3/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
import csv

In [5]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [6]:
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("He left after cleaning up.")
output

{'verbs': [{'verb': 'left',
   'description': '[ARG0: He] [V: left] [ARGM-TMP: after cleaning up] .',
   'tags': ['B-ARG0', 'B-V', 'B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'O']},
  {'verb': 'cleaning',
   'description': '[ARG0: He] left after [V: cleaning] up .',
   'tags': ['B-ARG0', 'O', 'O', 'B-V', 'O', 'O']}],
 'words': ['He', 'left', 'after', 'cleaning', 'up', '.']}

In [7]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [8]:
d = ["He left after cleaning up."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'left',
     'description': '[ARG0: He] [V: left] [ARGM-TMP: after cleaning up] .',
     'tags': ['B-ARG0', 'B-V', 'B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'O']},
    {'verb': 'cleaning',
     'description': '[ARG0: He] left after [V: cleaning] up .',
     'tags': ['B-ARG0', 'O', 'O', 'B-V', 'O', 'O']}],
   'words': ['He', 'left', 'after', 'cleaning', 'up', '.']}],
 array([1.]))

In [9]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [10]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [11]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [13]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} left after cleaning up.", meta=True,nsamples=150, remove_duplicates=True)

In [14]:
with open('/datasets/both_intransitive_transitive_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/both_intransitive_transitive_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 150 examples
Test cases:      150
Fails (rate):    1 (0.7%)

Example fails:
[ARG1: Julia] [V: left] [ARGM-TMP: after cleaning up] .
----
['Maria', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Ray', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Kenneth', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Harold', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Kate', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Kelly', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Elaine', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Anthony', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Patrick', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Louise', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Julia', 'left', 'after', 'cleaning', 'up', '.'] [False]
['Joseph', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Grace', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Nancy', 'left', 'after', 'cleaning', 'up', '.'] [ True]
['Diana', 'left', 'after', 'clean

In [12]:
d = ["He left the gift on the table."]
pred = predict_and_conf(d)
pred


([{'verbs': [{'verb': 'left',
     'description': '[ARG0: He] [V: left] [ARG1: the gift] [ARG2: on the table] .',
     'tags': ['B-ARG0',
      'B-V',
      'B-ARG1',
      'I-ARG1',
      'B-ARG2',
      'I-ARG2',
      'I-ARG2',
      'O']}],
   'words': ['He', 'left', 'the', 'gift', 'on', 'the', 'table', '.']}],
 array([1.]))

In [17]:
objects = ['the gift', 'the book', 'the keys', 'the wallet', 'the phone', 'the card']

In [18]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['objects']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1= Expect.single(found_arg1_people)

In [21]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("He left {objects} on the table.", objects = objects, meta=True,nsamples=150, remove_duplicates=True)
# initialize a test object
with open('/datasets/both_intransitive_transitive2_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/both_intransitive_transitive2_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 150 examples
Test cases:      150
Fails (rate):    150 (100.0%)

Example fails:
[ARG0: He] [V: left] [ARG1: the gift] [ARG2: on the table] .
----
[ARG0: He] [V: left] [ARG1: the phone] [ARGM-LOC: on the table] .
----
[ARG0: He] [V: left] [ARG1: the book] [ARG2: on the table] .
----
['He', 'left', 'the', 'gift', 'on', 'the', 'table', '.'] [False]
['He', 'left', 'the', 'wallet', 'on', 'the', 'table', '.'] [False]
['He', 'left', 'the', 'phone', 'on', 'the', 'table', '.'] [False]
['He', 'left', 'the', 'card', 'on', 'the', 'table', '.'] [False]
['He', 'left', 'the', 'phone', 'on', 'the', 'table', '.'] [False]
['He', 'left', 'the', 'card', 'on', 'the', 'table', '.'] [False]
['He', 'left', 'the', 'wallet', 'on', 'the', 'table', '.'] [False]
['He', 'left', 'the', 'phone', 'on', 'the', 'table', '.'] [False]
['He', 'left', 'the', 'wallet', 'on', 'the', 'table', '.'] [False]
['He', 'left', 'the', 'book', 'on', 'the', 'table', '.'] [False]
['He', 'left', 'the', 'book', 'on', 'the', 'tab

# Transitive verb

In [17]:
d = ["She loves cats."]
pred = predict_and_conf(d)
pred


([{'verbs': [{'verb': 'loves',
     'description': '[ARG0: She] [V: loves] [ARG1: cats] .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
   'words': ['She', 'loves', 'cats', '.']}],
 array([1.]))

In [18]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [19]:

def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [20]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [21]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} loves cats.", meta=True,nsamples=150, remove_duplicates=True)
# initialize a test object
with open('/datasets/transitive_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/transitive_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 150 examples
Test cases:      150
Fails (rate):    0 (0.0%)
['Emma', 'loves', 'cats', '.'] [ True]
['Andrea', 'loves', 'cats', '.'] [ True]
['Alfred', 'loves', 'cats', '.'] [ True]
['William', 'loves', 'cats', '.'] [ True]
['Lawrence', 'loves', 'cats', '.'] [ True]
['Jane', 'loves', 'cats', '.'] [ True]
['Frank', 'loves', 'cats', '.'] [ True]
['Elaine', 'loves', 'cats', '.'] [ True]
['Philip', 'loves', 'cats', '.'] [ True]
['Robert', 'loves', 'cats', '.'] [ True]
['Billy', 'loves', 'cats', '.'] [ True]
['Lawrence', 'loves', 'cats', '.'] [ True]
['Larry', 'loves', 'cats', '.'] [ True]
['Mary', 'loves', 'cats', '.'] [ True]
['Lauren', 'loves', 'cats', '.'] [ True]
['Sam', 'loves', 'cats', '.'] [ True]
['Angela', 'loves', 'cats', '.'] [ True]
['Claire', 'loves', 'cats', '.'] [ True]
['Arthur', 'loves', 'cats', '.'] [ True]
['Andrew', 'loves', 'cats', '.'] [ True]
['Ray', 'loves', 'cats', '.'] [ True]
['Annie', 'loves', 'cats', '.'] [ True]
['Jane', 'loves', 'cats', '.'] [ True]

## Intransive verbs

In [22]:
d = ["She sang."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'sang',
     'description': '[ARG0: She] [V: sang] .',
     'tags': ['B-ARG0', 'B-V', 'O']}],
   'words': ['She', 'sang', '.']}],
 array([1.]))

In [23]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [24]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [25]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [26]:
adjectives = ['loud', 'nice', 'emotional', 'horrible', 'bad', 'decent', 'good']

In [34]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} sang.", meta=True,nsamples=20, remove_duplicates = True)
# initialize a test object
with open('/datasets/intransitive_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/intransitive_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 20 examples
Test cases:      20
Fails (rate):    0 (0.0%)
['James', 'sang', '.'] [ True]
['Alan', 'sang', '.'] [ True]
['Wendy', 'sang', '.'] [ True]
['Christopher', 'sang', '.'] [ True]
['Louise', 'sang', '.'] [ True]
['Benjamin', 'sang', '.'] [ True]
['Sharon', 'sang', '.'] [ True]
['Evelyn', 'sang', '.'] [ True]
['Jeff', 'sang', '.'] [ True]
['Roger', 'sang', '.'] [ True]
['Sally', 'sang', '.'] [ True]
['John', 'sang', '.'] [ True]
['Colin', 'sang', '.'] [ True]
['Jason', 'sang', '.'] [ True]
['Martha', 'sang', '.'] [ True]
['Joan', 'sang', '.'] [ True]
['Joseph', 'sang', '.'] [ True]
['Lynn', 'sang', '.'] [ True]
['Nick', 'sang', '.'] [ True]
['Gary', 'sang', '.'] [ True]


In [35]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} sang very {adj}.", adj = adjectives, meta=True,nsamples=200, remove_duplicates = True)
with open('/datasets/intransitive_adj_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/intransitive_adj_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 200 examples
Test cases:      200
Fails (rate):    1 (0.5%)

Example fails:
[ARGM-DIS: Andrea] [V: sang] [ARG1: very nice] .
----
['Alexandra', 'sang', 'very', 'decent', '.'] [ True]
['Annie', 'sang', 'very', 'decent', '.'] [ True]
['Pamela', 'sang', 'very', 'nice', '.'] [ True]
['Dan', 'sang', 'very', 'loud', '.'] [ True]
['Judy', 'sang', 'very', 'decent', '.'] [ True]
['Walter', 'sang', 'very', 'nice', '.'] [ True]
['Richard', 'sang', 'very', 'bad', '.'] [ True]
['Lawrence', 'sang', 'very', 'decent', '.'] [ True]
['Steven', 'sang', 'very', 'decent', '.'] [ True]
['Patrick', 'sang', 'very', 'emotional', '.'] [ True]
['Edward', 'sang', 'very', 'loud', '.'] [ True]
['Simon', 'sang', 'very', 'nice', '.'] [ True]
['Lauren', 'sang', 'very', 'decent', '.'] [ True]
['Jack', 'sang', 'very', 'horrible', '.'] [ True]
['Sharon', 'sang', 'very', 'loud', '.'] [ True]
['Ellen', 'sang', 'very', 'nice', '.'] [ True]
['Patrick', 'sang', 'very', 'decent', '.'] [ True]
['Brian', 'sang', 'very

# End of Notebook